In [74]:
!pip install pandas mecab-python3 ipadic scikit-learn matplotlib seaborn openai


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [94]:
!pip install rank_bm25 sentence-transformers

   ---------------------------------------- 0.0/553.3 kB ? eta -:--:--
   ---------------------------------------- 553.3/553.3 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
    --------------------------------------- 1.8/113.8 MB 8.4 MB/s eta 0:00:14
   - -------------------------------------- 3.4/113.8 MB 8.4 MB/s eta 0:00:14
   - -------------------------------------- 5.2/113.8 MB 8.4 MB/s eta 0:00:13
   -- ------------------------------------- 6.6/113.8 MB 7.8 MB/s eta 0:00:14
   -- ------------------------------------- 8.1/113.8 MB 7.7 MB/s eta 0:00:14
   --- ------------------------------------ 9.7/113.8 MB 7.7 MB/s eta 0:00:14
   --- ------------------------------------ 11.0/113.8 MB 7.4 MB/s eta 0:00:14
   ---- ----------------------------------- 12.8/113.8 MB 7.5 MB/s eta 0:00:14
   ---- ----------------------------------- 13.9/113.8 MB 7.5 MB/s eta 0:00:14
   ----- ---------------------------------- 15.7/113.8 MB 7.4 MB/s eta 0:


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [115]:
import numpy as np
import pandas as pd
import MeCab
import ipadic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import re
import japanize_matplotlib
import os
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [116]:
base_df = pd.read_csv(r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Data\base_stories.tsv', sep='\t')
practice_df = pd.read_csv(r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Data\fiction_stories_practice.tsv', sep='\t')
# --- 1. テストデータの読み込み ---
test_df = pd.read_csv(r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Data\fiction_stories_test.tsv', sep='\t')

In [77]:
# MeCabの初期化 (ipadicを使用)
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)

In [78]:
def extract_nouns(text):
    """文章から名詞・固有名詞を抽出する"""
    if pd.isna(text): return []
    node = tagger.parseToNode(text)
    nouns = []
    while node:
        features = node.feature.split(',')
        # 名詞かつ（一般、固有名詞、サ変接続）を対象とする
        if features[0] == '名詞' and features[1] in ['一般', '固有名詞', 'サ変接続']:
            if len(node.surface) > 1: # 1文字の記号などは除外
                nouns.append(node.surface)
        node = node.next
    return nouns

In [79]:
base_df.columns

Index(['id', 'category', 'title', 'story'], dtype='object')

In [80]:
print(base_df.shape)
print(practice_df.shape)

(50, 4)
(20, 5)


In [81]:
practice_df.head()

,id_a,id_b,title_a,title_b,story
0,29,23,新幹線大爆破(1975),フルメタル・ジャケット,大都市で相次いだ爆発により通信と電力が断たれ、交通網と物流は停止する。インフラの弱点が露わに...
1,3,35,プライベート・ライアン,七人の侍,泥に沈む前線で、崩壊寸前の共同体を守る部隊に、敵地に取り残された通信員の救出命令が下る。だが...
2,11,43,ジョーカー,FIRST SLAM DUNK,景気後退が続き失業者が増える街で、主人公は過去の事故が原因で夢を諦め、家族とも距離を置いてい...
3,19,5,マイノリティ・リポート,インターステラー,重力異常が連鎖する未知の惑星へ、調査隊は自動運転車両で降下する。基地では「将来の映像を分析す...
4,2,19,マトリックス,マイノリティ・リポート,街中や店の天井に据え付けられた小型の撮像装置が、人々の行動を絶えず記録する社会。集まった映像...


In [82]:
# base_storiesの各作品からキーワードを抽出
base_df['keywords'] = base_df['story'].apply(extract_nouns)

In [83]:
base_df[['title', 'story', 'keywords']].head()

,title,story,keywords
0,スター・ウォーズ エピソードIV/新たなる希望,銀河規模の専制国家が宇宙を支配し、巨大な軍事基地で各地の星々を脅している時代。反対勢力は地下...,"[銀河, 規模, 専制, 国家, 宇宙, 支配, 軍事, 基地, 各地, 時代, 反対, 勢..."
1,マトリックス,昼は会社員、夜はハッカーとして生きるトーマス・アンダーソンは、自分の暮らす世界に強い違和感を...,"[会社, ハッカー, トーマス, アンダーソン, 自分, 世界, 違和感, 高層, ビル, ..."
2,プライベート・ライアン,第二次世界大戦末期のヨーロッパ戦線。海を渡って上陸作戦が始まり、兵士たちは鉄の障害物が並ぶ海...,"[世界, 大戦, 末期, ヨーロッパ, 戦線, 上陸, 作戦, 兵士, 障害, 海岸, 上陸..."
3,インセプション,近未来の企業社会では、人の意識に入り込んで情報を盗む犯罪が問題になっている。主人公のドム・コ...,"[未来, 企業, 社会, 意識, 情報, 犯罪, 主人公, ドム・コブ, 装置, 眠り, 共..."
4,インターステラー,地球の近未来。異常気象と病害で作物が次々に枯れ、食料不足と砂嵐が日常になる。人々は宇宙開発へ...,"[地球, 未来, 気象, 病害, 作物, 食料, 不足, 砂嵐, 日常, 人々, 宇宙, 開..."


In [84]:
def score_keyword_retention(row, base_df):
    """
    合成されたあらすじの中に、元ネタ(id_a, id_b)の単語が
    「そのまま」何％含まれているかを計算する
    """
    practice_story = row['story']
    results = {}
    
    for suffix in ['a', 'b']:
        target_id = row[f'id_{suffix}']
        # 元ネタのキーワード集合
        target_keywords = set(base_df[base_df['id'] == target_id]['keywords'].iloc[0])
        
        if not target_keywords:
            results[f'retention_{suffix}'] = 0
            continue
            
        # practiceの文章中に含まれているかチェック
        found_count = sum(1 for word in target_keywords if word in practice_story)
        results[f'retention_{suffix}'] = found_count / len(target_keywords)
        
    return pd.Series(results)

In [85]:
# スコアリング実行
retention_df = practice_df.apply(lambda r: score_keyword_retention(r, base_df), axis=1)
analysis_df = pd.concat([practice_df[['id_a', 'id_b']], retention_df], axis=1)

In [86]:
print("--- キーワード残存率分析 ---")
print(analysis_df[['retention_a', 'retention_b']].describe())

--- キーワード残存率分析 ---
       retention_a  retention_b
count    20.000000    20.000000
mean      0.110434     0.118066
std       0.039437     0.047186
min       0.049383     0.048077
25%       0.081381     0.073279
50%       0.108613     0.119117
75%       0.133112     0.152098
max       0.185185     0.227273


In [87]:
# 可視化
# analysis_df[['retention_a', 'retention_b']].plot(kind='box', title='Keyword Retention Rate')
# plt.ylabel('Retention Rate')
# plt.show()

In [88]:
vectorizer = TfidfVectorizer(tokenizer=extract_nouns, token_pattern=None)
tfidf_matrix = vectorizer.fit_transform(base_df['story'])
feature_names = vectorizer.get_feature_names_out()

In [89]:
def analyze_rare_word_survival(row, base_df, tfidf_matrix, feature_names, top_n=10):
    results = []
    for suffix in ['a', 'b']:
        target_id = row[f'id_{suffix}']
        idx = base_df[base_df['id'] == target_id].index[0]
        
        # TF-IDF上位N個の「その作品らしい単語」を抽出
        row_data = tfidf_matrix.getrow(idx).toarray().flatten()
        top_indices = row_data.argsort()[-top_n:]
        rare_words = [feature_names[i] for i in top_indices]
        
        # 合成文に含まれているか
        found = [w for w in rare_words if w in row['story']]
        results.append(len(found) / top_n)
    return pd.Series(results, index=['rare_retention_a', 'rare_retention_b'])

In [90]:
rare_analysis = practice_df.apply(lambda r: analyze_rare_word_survival(r, base_df, tfidf_matrix, feature_names), axis=1)

In [91]:
print("--- レア単語（特徴語）の生存率 ---")
print(rare_analysis.describe())

--- レア単語（特徴語）の生存率 ---
       rare_retention_a  rare_retention_b
count          20.00000         20.000000
mean            0.16500          0.200000
std             0.11821          0.152177
min             0.00000          0.000000
25%             0.10000          0.100000
50%             0.20000          0.150000
75%             0.20000          0.300000
max             0.50000          0.600000


In [92]:
def plot_story_flow(sample_idx, save_dir="story_flows"):
    # 保存用ディレクトリの作成
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    row = practice_df.iloc[sample_idx]
    sentences = [s for s in re.split(r'(?<=。)', row['story']) if s]
    
    # ベクトル計算
    sent_vectors = vectorizer.transform([" ".join(extract_nouns(s)) for s in sentences])
    sims = cosine_similarity(sent_vectors, tfidf_matrix)
    
    plt.figure(figsize=(10, 4))
    
    # 正解の2作品への類似度推移
    idx_a = base_df[base_df['id'] == row['id_a']].index[0]
    idx_b = base_df[base_df['id'] == row['id_b']].index[0]
    
    plt.plot(sims[:, idx_a], marker='o', label=f"Target A (ID:{row['id_a']})", color='blue')
    plt.plot(sims[:, idx_b], marker='s', label=f"Target B (ID:{row['id_b']})", color='red')
    
    # ノイズの最大値
    mask = np.ones(sims.shape[1], dtype=bool)
    mask[[idx_a, idx_b]] = False
    max_noise = sims[:, mask].max(axis=1)
    plt.plot(max_noise, label="Max Noise", color='gray', linestyle='--', alpha=0.5)
    
    plt.title(f"Story Flow Analysis: Practice ID {sample_idx}")
    plt.xlabel("Sentence Position")
    plt.ylabel("Similarity Score")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # --- 保存処理 ---
    save_path = os.path.join(save_dir, f"flow_{sample_idx:03d}.png")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close() # メモリ解放のためにcloseする
    print(f"Saved: {save_path}")

In [93]:
from pathlib import Path

# 文字列をPathオブジェクトに変換
output_path = Path(r"C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot")

for i in range(20):
    plot_story_flow(i, save_dir=str(output_path))

Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_000.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_001.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_002.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_003.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_004.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_005.png
Saved: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\residual_plot\flow_006.png
Saved: C:\Users\管理\Documents\GitHu

In [96]:
# --- 1. BM25 の準備 ---
# base_stories の全ストーリーをトークン化（名詞抽出）してインデックス作成
tokenized_base = [extract_nouns(s) for s in base_df['story']]
bm25 = BM25Okapi(tokenized_base)

In [97]:
# --- 2. ベクトル検索 (Sentence-Transformers) の準備 ---
# 日本語に強く、軽量なモデルを使用します
model_name = 'intfloat/multilingual-e5-small'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\管理\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\管理\.cache\huggingface\hub\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [98]:
# base_stories をベクトル化 (E5モデルの形式に従い "passage: " を付与)
base_texts = ["passage: " + s for s in base_df['story']]
base_embeddings = model.encode(base_texts, normalize_embeddings=True)

In [99]:
def get_sliding_windows(text, window_size=2):
    """文章を指定した文数の窓で分割する"""
    sentences = [s.strip() + "。" for s in re.split(r'(?<=。)', text) if s.strip()]
    if len(sentences) <= window_size:
        return [text]
    
    windows = []
    for i in range(len(sentences) - window_size + 1):
        windows.append("".join(sentences[i:i + window_size]))
    return windows

In [100]:
def hybrid_window_search(query_story, k=60):
    """
    窓ごとに BM25 と Vector 検索を行い、RRF で統合。
    各作品について、全窓の中での『最高順位』を最終的な順位として採用する。
    """
    windows = get_sliding_windows(query_story, window_size=2) # 2文ずつの窓
    num_docs = len(base_df)
    
    # 各作品ID(index)ごとの最小順位（最高順位）を保持する
    # 初期値は最下位（順位 = ドキュメント数）
    best_bm25_ranks = np.full(num_docs, num_docs)
    best_vec_ranks = np.full(num_docs, num_docs)

    for window in windows:
        # --- BM25 検索 ---
        q_tokens = extract_nouns(window)
        if q_tokens:
            bm25_scores = bm25.get_scores(q_tokens)
            # 順位に変換 (値が大きい順に 1, 2, 3...)
            ranks = np.argsort(np.argsort(bm25_scores)[::-1]) + 1
            best_bm25_ranks = np.minimum(best_bm25_ranks, ranks)

        # --- Vector 検索 ---
        q_emb = model.encode(["query: " + window], normalize_embeddings=True)
        vec_sims = cosine_similarity(q_emb, base_embeddings)[0]
        # 順位に変換
        ranks = np.argsort(np.argsort(vec_sims)[::-1]) + 1
        best_vec_ranks = np.minimum(best_vec_ranks, ranks)

    # --- RRF (Reciprocal Rank Fusion) による統合スコアリング ---
    # 各作品が、いずれかの窓で「BM25上位」または「Vector上位」に入っていれば高スコアになる
    rrf_scores = (1.0 / (k + best_bm25_ranks)) + (1.0 / (k + best_vec_ranks))
    
    # スコア順にソートした結果を返す
    top_indices = np.argsort(rrf_scores)[::-1]
    
    results = base_df.iloc[top_indices].copy()
    results['search_score'] = rrf_scores[top_indices]
    return results

In [101]:
def evaluate_search(df, top_n=10):
    hits = 0
    total_relevant = len(df) * 2
    
    for _, row in df.iterrows():
        preds = hybrid_window_search(row['story'])
        top_ids = preds['id'].head(top_n).values
        
        # 正解が含まれているかチェック
        if row['id_a'] in top_ids: hits += 1
        if row['id_b'] in top_ids: hits += 1
            
    recall_at_n = hits / total_relevant
    print(f"Recall@{top_n}: {recall_at_n:.2%} ({hits}/{total_relevant})")
    return recall_at_n

In [102]:
# 練習用データ 20件で検証
print("Evaluating on practice_df...")
recall = evaluate_search(practice_df, top_n=10)

Evaluating on practice_df...
Recall@10: 85.00% (34/40)


In [105]:
def check_search_details(df, top_n=10):
    detailed_results = []
    
    for i, row in df.iterrows():
        preds = hybrid_window_search(row['story'])
        # 全作品の中での正解の順位を取得 (indexは0始まりなので+1する)
        rank_a = np.where(preds['id'].values == row['id_a'])[0][0] + 1
        rank_b = np.where(preds['id'].values == row['id_b'])[0][0] + 1
        
        detailed_results.append({
            'practice_id': i,
            'id_a': row['id_a'],
            'rank_a': rank_a,
            'hit_a': rank_a <= top_n,
            'id_b': row['id_b'],
            'rank_b': rank_b,
            'hit_b': rank_b <= top_n,
        })
    
    return pd.DataFrame(detailed_results)

In [106]:
# 実行
check_df = check_search_details(practice_df)

In [110]:
# 1. データの抽出
missed_cases = check_df[(check_df['rank_a'] > 10) | (check_df['rank_b'] > 10)]

# 2. 画面表示（Jupyter上で表が見えるようになります）
print("--- 正解が10位以内に入らなかったケース ---")
display(missed_cases)

--- 正解が10位以内に入らなかったケース ---


,practice_id,id_a,rank_a,hit_a,id_b,rank_b,hit_b
4,4,2,28,False,19,1,True
7,7,24,4,True,18,26,False
8,8,13,13,False,28,20,False
10,10,41,1,True,10,11,False
13,13,40,9,True,50,14,False


In [111]:
# 3. CSVへの保存（missed_cases に対して実行する）
save_path = r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\accuracy_not_10.csv'
missed_cases.to_csv(save_path, index=False) # index=False をつけると余計なID列が入りません

In [112]:
# 全体の順位の分布を確認
print("\n--- 正解順位の統計 ---")
print(check_df[['rank_a', 'rank_b']].describe())


--- 正解順位の統計 ---
          rank_a     rank_b
count  20.000000  20.000000
mean    4.300000   5.550000
std     6.366855   7.029824
min     1.000000   1.000000
25%     1.000000   1.000000
50%     2.000000   2.500000
75%     4.000000   6.500000
max    28.000000  26.000000


In [113]:
Statistics_on_Correct_Answer_Rankings = check_df[['rank_a', 'rank_b']].describe()

In [114]:
# 3. CSVへの保存（missed_cases に対して実行する）
save_path = r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\Statistics_on_Correct_Answer_Rankings.csv'
Statistics_on_Correct_Answer_Rankings.to_csv(save_path, index=False) # index=False をつけると余計なID列が入りません

In [117]:
# --- 2. 予測の実行 ---
# ※ 340件の処理には数分かかる場合があります
results = []

print("Starting predictions for test data...")
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    test_id = row['id']
    story_text = row['story']
    
    # 前のステップで作成したハイブリッド検索を実行
    search_results = hybrid_window_search(story_text)
    
    # 上位2件の作品IDを取得
    # 本来はここで検索上位10~30件を LLM に渡し、2つに絞り込むリランキングを行う
    top_2_ids = search_results['id'].head(2).values
    
    # 戦略に基づき、予測した2つのIDを昇順（小さい順）に並べ替え
    pred1, pred2 = sorted(top_2_ids)
    
    results.append([test_id, pred1, pred2])

Starting predictions for test data...


100%|██████████| 340/340 [00:51<00:00,  6.61it/s]


In [119]:
submit_df = pd.DataFrame(results, columns=['ID', 'Pred1', 'Pred2'])

In [120]:
output_filename = r'C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\submission.csv'
submit_df.to_csv(output_filename, index=False, header=False)

In [124]:
print(f"\nSuccessfully created: {output_filename}")
submit_df.head()


Successfully created: C:\Users\管理\Documents\GitHub\Data-Analysis_competition\Analysis\sigante_anime\Analysis_file_3rd\out_put\submission.csv


,ID,Pred1,Pred2
0,1,26,27
1,2,8,34
2,3,33,37
3,4,26,36
4,5,17,18


In [125]:
def calculate_metrics(df, top_n=10):
    perfect_matches = 0  # 2つとも正解（順不同）
    partial_matches = 0  # 1つだけ正解
    both_in_top_n = 0    # 2つとも上位N件に入っている
    any_in_top_n = 0     # 少なくとも1つが上位N件に入っている
    
    total_cases = len(df)
    
    print(f"Calculating metrics for {total_cases} cases...")
    
    for _, row in df.iterrows():
        # 検索の実行
        preds = hybrid_window_search(row['story'])
        top_ids = preds['id'].values
        
        # 1. 完全一致の判定 (Top 2 が正解ペアと一致するか)
        predicted_top_2 = set(top_ids[:2])
        true_ids = {row['id_a'], row['id_b']}
        
        if predicted_top_2 == true_ids:
            perfect_matches += 1
        
        # 1つだけ正解しているか（デバッグ用）
        match_count_top_2 = len(predicted_top_2.intersection(true_ids))
        if match_count_top_2 == 1:
            partial_matches += 1

        # 2. 検索エンジンの性能判定 (Top 10 に正解が含まれているか)
        search_top_n = set(top_ids[:top_n])
        match_count_top_n = len(search_top_n.intersection(true_ids))
        
        if match_count_top_n == 2:
            both_in_top_n += 1
        if match_count_top_n >= 1:
            any_in_top_n += 1

    # 指標の計算
    accuracy = perfect_matches / total_cases
    recall_both = both_in_top_n / total_cases
    
    print("\n--- 評価結果 ---")
    print(f"1. 完全一致正解率 (Perfect Match Accuracy): {accuracy:.2%} ({perfect_matches}/{total_cases})")
    print(f"   ※ 上位2件が正解の2つと完全に一致した割合")
    print(f"\n2. Top-{top_n} 両方包含率 (Both in Top-{top_n}): {recall_both:.2%} ({both_in_top_n}/{total_cases})")
    print(f"   ※ 上位{top_n}件の中に、正解の2つが両方とも含まれていた割合")
    print(f"\n3. 参考指標:")
    print(f"   - 少なくとも1つ正解 (Top 2): {(perfect_matches + partial_matches)/total_cases:.2%}")
    print(f"   - 少なくとも1つ包含 (Top {top_n}): {any_in_top_n/total_cases:.2%}")

In [126]:
# 実行
calculate_metrics(practice_df)

Calculating metrics for 20 cases...

--- 評価結果 ---
1. 完全一致正解率 (Perfect Match Accuracy): 30.00% (6/20)
   ※ 上位2件が正解の2つと完全に一致した割合

2. Top-10 両方包含率 (Both in Top-10): 75.00% (15/20)
   ※ 上位10件の中に、正解の2つが両方とも含まれていた割合

3. 参考指標:
   - 少なくとも1つ正解 (Top 2): 80.00%
   - 少なくとも1つ包含 (Top 10): 95.00%
